# Step 5 - Applying weighted distance calculations
## Project: Data-driven bicycle networks

This notebook is embedding bicycle counts onto the street network of Copenhagen

## Preliminaries

### Parameters

In [1]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

Loaded parameters.



### Setup

In [ ]:
%run -i path.py
#%run -i setup.py
%run -i setupCPH.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [3]:
%run -i functions.py

Loaded functions.



### Load street network graph from Copenhagen

In [ ]:
G_carall = ox.load_graphml('../../bikenwgrowth_external/data/copenhagen/bikedata/G_carall_graphml.graphml')

## Calculating weighted distances and applying to the street network of Copenhagen

The formula for weighted distances d_W is being utilized

In [ ]:
# alpha zero is pop_den_attr
#aplha_one is bcount_attr
#aplha_half is bcount_pop_den_attr

length = list(nx.get_edge_attributes(G_carall, 'length').values())
bcount = list(nx.get_edge_attributes(G_carall, 'bcount').values())
pop_den = list(nx.get_edge_attributes(G_carall, 'pop_den').values())

#Normalise values and convert to fraction
length_norm = normalise_list(length.copy())
bcount_frac_norm = normalise_list([1/bcount[i] for i in range(len(bcount))])
pop_den_frac_norm = normalise_list([1/pop_den[i] for i in range(len(pop_den))])

#create df with all attributes
edge_attributes = pd.DataFrame({'length':lengths, 'length_norm':length_norm, 'bcount_frac': bcount_frac_norm, 'pop_den_frac': pop_den_frac_norm})

#create weighted distance measures and normalise these
edge_attributes['alpha_zero'] = normalise_list(0* (edge_attributes['length']*edge_attributes['bcount_frac']) + (1-0)*(edge_attributes['length']*edge_attributes['pop_den_frac']))
edge_attributes['alpha_half'] = normalise_list(0.5* (edge_attributes['length']*edge_attributes['bcount_frac']) + (0.5)*(edge_attributes['length']*edge_attributes['pop_den_frac']))
edge_attributes['alpha_one'] = normalise_list(1* (edge_attributes['length']*edge_attributes['bcount_frac']) + (0)*(edge_attributes['length']*edge_attributes['pop_den_frac']))

# apply to G_carall graph as attributes
i=-1
for e in G_carall.edges():
    i+=1
    a,b=e
    length_norm = edge_attributes['length_norm'].iloc[i]
    alpha_zero = edge_attributes['alpha_zero'].iloc[i]
    alpha_half = edge_attributes['alpha_half'].iloc[i]
    alpha_one = edge_attributes['alpha_one'].iloc[i]
    nx.set_edge_attributes(G_carall, {(a,b,0): {"length_norm": length_norm}})
    nx.set_edge_attributes(G_carall, {(a,b,0): {"pop_den_attr": alpha_zero}})
    nx.set_edge_attributes(G_carall, {(a,b,0): {"bcount_pop_den_attr": alpha_half}})
    nx.set_edge_attributes(G_carall, {(a,b,0): {"bcount_attr": alpha_one}})

#save graph
ox_to_csv(G_carall, PATH["data"] + 'copenhagen' + "/", 'copenhagen', 'carall')
ox_to_csv(ox.simplify_graph(G_carall), PATH["data"] + 'copenhagen' + "/", 'copenhagen', 'carall', "_simplified")
ox.save_graphml(G_carall, "../../bikenwgrowth_external/data/copenhagen/bikedata/G_carall_graphml.graphml")